# Cleaning procedure.

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [1]:
df = pd.read_csv(
            "/Users/edwardsujono/Python_Project/h1b_visa_analytic/data/h1b_kaggle.csv"
)

We need to delete WITHDRAWN status from the data collections. WITHDRAWN status means that particular user has withdrawn their visa request before the result is decided, so that kind of data is unrelevant to our analysis. 
Latitude and Longitude are not relevant to our analysis as well since the value is continuos.

In [2]:
df.drop(df[df.CASE_STATUS == "WITHDRAWN"].index, inplace=True)
df.drop("lon", 1, inplace=True)
df.drop("lat", 1, inplace=True)

Remove all the empty column

In [8]:
df = df.dropna()
df

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.00,2016.0,"ANN ARBOR, MICHIGAN"
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.00,2016.0,"PLANO, TEXAS"
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.00,2016.0,"JERSEY CITY, NEW JERSEY"
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.00,2016.0,"DENVER, COLORADO"
5,6,CERTIFIED-WITHDRAWN,BURGER KING CORPORATION,CHIEF EXECUTIVES,"EXECUTIVE V P, GLOBAL DEVELOPMENT AND PRESIDEN...",Y,225000.00,2016.0,"MIAMI, FLORIDA"
6,7,CERTIFIED-WITHDRAWN,BT AND MK ENERGY AND COMMODITIES,CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,91021.00,2016.0,"HOUSTON, TEXAS"
7,8,CERTIFIED-WITHDRAWN,"GLOBO MOBILE TECHNOLOGIES, INC.",CHIEF EXECUTIVES,CHIEF OPERATIONS OFFICER,Y,150000.00,2016.0,"SAN JOSE, CALIFORNIA"
8,9,CERTIFIED-WITHDRAWN,ESI COMPANIES INC.,CHIEF EXECUTIVES,PRESIDENT,Y,127546.00,2016.0,"MEMPHIS, TEXAS"
10,11,CERTIFIED-WITHDRAWN,H.J. HEINZ COMPANY,CHIEF EXECUTIVES,"CHIEF INFORMATION OFFICER, HEINZ NORTH AMERICA",Y,182978.00,2016.0,"PITTSBURGH, PENNSYLVANIA"
11,12,CERTIFIED-WITHDRAWN,DOW CORNING CORPORATION,CHIEF EXECUTIVES,VICE PRESIDENT AND CHIEF HUMAN RESOURCES OFFICER,Y,163717.00,2016.0,"MIDLAND, MICHIGAN"


Inner join with the real SOC data because several data in SOC_NAME is rubbish

In [11]:
df_soc = pd.read_csv(
            "/Users/edwardsujono/Python_Project/h1b_visa_analytic/data/soc_code.csv"
        )
result = pd.merge(df,  df_soc[["Code", "Title"]], left_on="SOC_NAME", right_on="Title", how="inner")
df = result
# result.to_csv("/Users/edwardsujono/Python_Project/h1b_visa_analytic/data/clean_data.csv")

Get all the CASE STATUS first

In [12]:
all_case_status = df.CASE_STATUS.unique()
all_case_status

array(['CERTIFIED-WITHDRAWN', 'CERTIFIED', 'DENIED', 'REJECTED'], dtype=object)

Basically we just interested with these 4 statuses: 'CERTIFIED-WITHDRAWN', 'CERTIFIED', 'DENIED', 'REJECTED', all the rest is unusefull.

In [13]:
df = df[df.CASE_STATUS.isin(['CERTIFIED-WITHDRAWN', 'CERTIFIED', 'DENIED', 'REJECTED'])]
df.CASE_STATUS.value_counts()

CERTIFIED              1352292
CERTIFIED-WITHDRAWN     100505
DENIED                   63568
REJECTED                     2
Name: CASE_STATUS, dtype: int64

# Visualization

On each year applicants that is certified increase

In [ ]:
%matplotlib inline
df_year_case_status = df[['CASE_STATUS', 'YEAR']]
ax=sns.factorplot('YEAR', data=df_year_case_status, hue='CASE_STATUS', kind='count', palette='PRGn')
sns.set(style="white", color_codes=True)

Next feature that we interested to investigate is the SOC_CODE, since it is the major deciding factor of any applicants

In [22]:
df.head(3)

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,Code,Title
0,1266531,CERTIFIED-WITHDRAWN,TEXAS STATE UNIVERSITY-SAN MARCOS,"Biological Scientists, All Other",POSTDOCTORAL RESEARCH ASSOCIATE,Y,43118.0,2014.0,"SAN MARCOS, TEXAS",19-1029.00,"Biological Scientists, All Other"
1,1266650,CERTIFIED-WITHDRAWN,INDIANA UNIV. PURDUE UNIV. INDIANAPOLIS,"Biological Scientists, All Other",POSTDOCTORAL RESEARCH ASSOCIATE,Y,33405.0,2014.0,"GARY, INDIANA",19-1029.00,"Biological Scientists, All Other"
2,1266656,CERTIFIED-WITHDRAWN,THE J. DAVID GLADSTONE INSTITUTES,"Biological Scientists, All Other",BIOINFORMATICS FELLOW,Y,46155.0,2014.0,"SAN FRANCISCO, CALIFORNIA",19-1029.00,"Biological Scientists, All Other"


In [12]:
df_year_soc_code = df[['YEAR', 'Code']]
df_year_soc_code.Code.value_counts()

15-1121.00    284982
15-1131.00    220764
15-1132.00    187537
15-1199.00     67990
15-1133.00     43183
13-1111.00     37688
13-2051.00     33056
13-2011.00     29946
17-2141.00     25473
17-2072.00     22468
17-2071.00     21447
13-1161.00     21419
15-1141.00     21367
29-1069.00     21213
15-2031.00     18634
15-1142.00     17957
19-1042.00     16254
11-3021.00     15467
29-1123.00     14284
19-1021.00     13895
17-2112.00     12227
11-2021.00     10247
17-2051.00      9651
19-1029.00      9611
15-2041.00      8859
29-1063.00      8825
19-2031.00      8350
27-1024.00      7790
11-1021.00      7100
29-1051.00      6967
               ...  
43-4141.00         1
27-2042.01         1
51-3092.00         1
37-2012.00         1
51-3023.00         1
37-2021.00         1
39-4021.00         1
47-5021.00         1
31-9094.00         1
29-2054.00         1
47-5013.00         1
49-9061.00         1
51-6011.00         1
51-4199.00         1
47-2011.00         1
49-3041.00         1
51-9198.00   

In [44]:
%matplotlib inline
df_year_soc_code = df[['YEAR', 'Code']]
tops = df_year_soc_code.Code.value_counts().index.tolist()[:5]
# print (tops)
df_year_soc_code  = df_year_soc_code[df_year_soc_code.Code.isin(tops)]
df_year_soc_code.YEAR.unique()
ax=sns.factorplot('YEAR', data=df_year_soc_code, hue='Code', kind='count')
sns.set(style="white", color_codes=True)

array([ 2014.,  2013.,  2012.,  2011.])

Wahh !!!, it is so interesting !!, all the tops job can acquire the Visa just on 2014 downward.
Then we need to extract what job 